In [2]:
#get the probabilities

In [3]:
import pandas as pd
import os
import re
from nltk.stem.porter import *
import math

In [4]:
data = pd.read_csv("Review_Subset.csv")

In [5]:
def getNgrams(text, n):
    if n == 1:
        return text
    
    text.append("<end>")
    ngramList = []
    for i in range(len(text) - (n-1)):
        ngram = text[i:i+n]
        ngramList.append(ngram)
    
    return ngramList

In [6]:
unigramDict = {}
bigramDict = {}
stemmer = PorterStemmer()

def getCountsinRow(row):
    text = row["text"]
    parsedText = re.findall("(\w+)", text)
    
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        
    unigram = getNgrams(parsedText, 1)
    bigram = getNgrams(parsedText, 2)
    ngrams = (unigram,bigram)
    for index in range(len(ngrams)):
        for eachWord in ngrams[index]:
            
            if index == 0:
                #unigram
                
                if eachWord in unigramDict:
                    unigramDict[eachWord] +=1
                else:
                    unigramDict[eachWord] =1
                    
            if index == 1:
                #bigram
                    eachWord = tuple(eachWord)
                    if eachWord in bigramDict:
                        bigramDict[eachWord] +=1
                    else:
                        bigramDict[eachWord] =1

            

In [7]:

#get the probabilies into unigramsDict and bigramsDict
data.apply(getCountsinRow, axis = 1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
1970    None
1971    None
1972    None
1973    None
1974    None
1975    None
1976    None
1977    None
1978    None
1979    None
1980    None
1981    None
1982    None
1983    None
1984    None
1985    None
1986    None
1987    None
1988    None
1989    None
1990    None
1991    None
1992    None
1993    None
1994    None
1995    None
1996    None
1997    None
1998    None
1999    None
Length: 2000, dtype: object

In [8]:
print(len(unigramDict))
print(len(bigramDict))

6120
60028


# SAVE UNIGRAM/BIGRAM FREQ. INTO A FILE

In [ ]:
#export unigrams
extractList = []
for eachKey in unigramDict:
    extractList.append([eachKey,unigramDict[eachKey]])

pd.DataFrame(extractList, columns = ["word", "count"]).to_csv("unigramProbabilities.csv", index = False)

In [ ]:
#export bigrams
extractList = []
for eachKey in bigramDict:
    extractList.append([eachKey[0],eachKey[1],bigramDict[eachKey]])
    
pd.DataFrame(extractList, columns = ["first", "second", "count"]).to_csv("bigramProbabilities.csv", index = False)

In [ ]:
print(data)

# UNIGRAM VECTOR MODEL

In [9]:
#get the vector model in data
unigramVM = []
def getUnigramReviewVectors(row):
    
    #we have data,unigrams,bigrams
    rowVector = {}
    text = row["text"]
    score = row["score"]
    parsedText = re.findall("(\w+)", text)
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        if parsedText[index] in rowVector:
            rowVector[parsedText[index]] +=1
        else:
            rowVector[parsedText[index]] = 1
    
    if score >= 4:
        rowVector["<label>"] = "good"
    else:
        rowVector["<label>"] = "bad"
        
    unigramVM.append(rowVector)
    

data.apply(getUnigramReviewVectors, axis = 1)
    
    
    

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
1970    None
1971    None
1972    None
1973    None
1974    None
1975    None
1976    None
1977    None
1978    None
1979    None
1980    None
1981    None
1982    None
1983    None
1984    None
1985    None
1986    None
1987    None
1988    None
1989    None
1990    None
1991    None
1992    None
1993    None
1994    None
1995    None
1996    None
1997    None
1998    None
1999    None
Length: 2000, dtype: object

# SAVE UVM

In [10]:
pd.DataFrame(unigramVM).to_csv("UnigramVectorModel.csv", index = False)

# GET TFIDF UNIGRAM MODEL

In [ ]:
TFIDFVM = []
unigramDF = pd.DataFrame(unigramVM)
def getTFIDFReviewVectors(row):
    
    #we have data,unigrams,bigrams
    rowVector = {}
    text = row["text"]
    score = row["score"]
    parsedText = re.findall("(\w+)", text)
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        TF = unigramDict[parsedText[index]]
        IDF = math.log2()
    
    if score > 4:
        rowVector["<label>"] = "good"
    else:
        rowVector["<label>"] = "bad"
        
    unigramVM.append(rowVector)
    



In [ ]:
unigramDF = pd.DataFrame(unigramVM)

In [ ]:
unigramDF.drop("<label>", inplace = True)
